# Classificação de animais com Pyspark/MLLib

In [35]:
from pyspark.sql import SparkSession

from pyspark.ml.classification import LinearSVC
from pyspark.ml.linalg import Vectors


spark = (
    SparkSession
    .builder
    .appName('classificação-animais')
    .getOrCreate()
)
spark

In [42]:
# features (1 sim, 0 não)
# pelo longo?
# perna curta?
# faz auau?
porco1 = [0, 1, 0]
porco2 = [0, 1, 1]
porco3 = [1, 1, 0]

cachorro1 = [0, 1, 1]
cachorro2 = [1, 0, 1]
cachorro3 = [1, 1, 1]

misterio1 = [1, 1 ,1]
misterio2 = [1, 1, 0]
misterio3 = [0, 1, 1]

In [38]:
# Criando o dataset de treino

data = [(1.0, Vectors.dense(porco1)),
        (1.0, Vectors.dense(porco2)),
        (1.0, Vectors.dense(porco3)),
        (0.0, Vectors.dense(cachorro1)),
        (0.0, Vectors.dense(cachorro2)),
        (0.0, Vectors.dense(cachorro3))
        ]
df_treino = spark.createDataFrame(data, ["label", "features"])

df_treino


label,features
1.0,"[0.0,1.0,0.0]"
1.0,"[0.0,1.0,1.0]"
1.0,"[1.0,1.0,0.0]"
0.0,"[0.0,1.0,1.0]"
0.0,"[1.0,0.0,1.0]"
0.0,"[1.0,1.0,1.0]"


In [17]:
# Criando o modelo

modelo = LinearSVC(
    featuresCol = 'features',
    labelCol = 'label'
).fit(df_treino)

modelo

LinearSVCModel: uid=LinearSVC_a9a55b28fa6d, numClasses=2, numFeatures=3

In [55]:
data_test = [
    (0, Vectors.dense(1, 1, 1)),
    (1, Vectors.dense(1, 1, 0)),
    (1, Vectors.dense(0, 1, 1)),
]

df_test = spark.createDataFrame(data_test, ["label", "features"])
df_test

label,features
0,"[1.0,1.0,1.0]"
1,"[1.0,1.0,0.0]"
1,"[0.0,1.0,1.0]"


In [59]:
previsao = modelo.transform(df_test)

previsao

label,features,rawPrediction,prediction
0,"[1.0,1.0,1.0]",[1.21210806892682...,0.0
1,"[1.0,1.0,0.0]",[-1.4463090443872...,1.0
1,"[0.0,1.0,1.0]",[0.38966043675672...,0.0


In [41]:
# Testando o modelo

print(modelo.predict(Vectors.dense(misterio1)) )
print(modelo.predict(Vectors.dense(misterio2)) )
print(modelo.predict(Vectors.dense(misterio3)) )

0.0
1.0
0.0
